In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# --- 1. The Physics Model ---
def cornell_potential_ansatz(r, A, sigma, B):
    """
    V(r) = A + sigma * r - B / r
    """
    return A + sigma * r - B / r

# --- 2. Data Preparation ---
# Your provided V(R) points
data_points = [
    (1.0, 0.33528340541297175), (2.0, 0.4848363896649771), 
    (3.0, 0.5677430149847106), (4.0, 0.6280987860250569), 
    (5.0, 0.6800002088341914), (6.0, 0.7280384617164685), 
    (7.0, 0.7753247910476052), (8.0, 0.8185376088350615)
]

rs, vs = zip(*data_points)
rs = np.array(rs)
vs = np.array(vs)
errs = np.full_like(vs, 0.001) # Assuming a small uniform error for fitting weights

# --- 3. The Fitting Logic ---
def calculate_r0(rs, vs, errs, target_force=1.65):
    # Initial Guesses
    sigma_guess = (vs[-1] - vs[0]) / (rs[-1] - rs[0])
    B_guess = 0.26
    A_guess = vs[0] - sigma_guess * rs[0] + (B_guess / rs[0])
    p0 = [A_guess, sigma_guess, B_guess]

    try:
        popt, _ = curve_fit(cornell_potential_ansatz, rs, vs, p0=p0, sigma=errs)
        A, sig, B = popt
        
        # Solving r^2 * dV/dr = 1.65 => r0 = sqrt((1.65 - B) / sigma)
        numerator = target_force - B
        if numerator < 0 or sig <= 0:
            return None, popt
        
        r0 = np.sqrt(numerator / sig)
        return r0, popt
    except Exception as e:
        print(f"Fit failed: {e}")
        return None, None

# --- 4. Execution and Visualization ---
r0_val, params = calculate_r0(rs, vs, errs)

if r0_val:
    A, sig, B = params
    print(f"--- Fit Results ---")
    print(f"r0/a:     {r0_val:.6f}")
    print(f"Sigma:    {sig:.6f}")
    print(f"B (Coul): {B:.6f}")
    print(f"A (Shift):{A:.6f}")

    # Plotting
    plt.figure(figsize=(8, 5))
    plt.scatter(rs, vs, color='red', label='Lattice Data $V(R)$')
    
    r_fine = np.linspace(rs[0], rs[-1], 100)
    plt.plot(r_fine, cornell_potential_ansatz(r_fine, *params), label='Cornell Fit')
    
    plt.axvline(x=r0_val, color='green', linestyle='--', label=f'$r_0 \\approx {r0_val:.2f}$')
    plt.xlabel('R/a')
    plt.ylabel('aV(R)')
    plt.title('Cornell Potential Fit to Extract Sommer Parameter $r_0$')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Could not calculate a valid r0.")

--- Cornell Potential Fit Results ---
r0/a:       3.0594
Sigma:      0.1484
B (Coul):   0.2607
A (Const):  -0.4903
Red. Chi2:  0.2094
